In [3]:
import numpy as np
# import h5py 
# import tables
from scipy.io import loadmat

In [6]:
# import sys
# #sys.path.append("..")
# sys.path.append("/Users/jan/Documents/")

In [4]:
data_dir = "/Users/jan/Documents/Studia/AdvancedProgramming/project/ToFFi_Toolbox-master/ToFFi_Toolbox-20211013/DATA_PREPARATION/HCP_DATA_PREP/output/000000_RENAMED"

atlas_dir = '/Users/jan/Documents/Studia/AdvancedProgramming/project/ToFFi_Toolbox-master/ToFFi_Toolbox-20211013/commonData/Schaefer2018_100Parcels_7Networks_ATLAS_interpolated_on_8mm_HCP_template.mat'

template_dir = '/Users/jan/Documents/Studia/AdvancedProgramming/project/ToFFi_Toolbox-master/ToFFi_Toolbox-20211013/commonData/templategrid_HCP_8mm.mat'

In [3]:
# %cd '/Users/jan/Documents/Studia/AdvancedProgramming/project/ToFFi_Toolbox-master/ToFFi_Toolbox-20211013/DATA_PREPARATION/HCP_DATA_PREP/output/000000_RENAMED/Sub_1'

/Users/jan/Documents/Studia/AdvancedProgramming/project/ToFFi_Toolbox-master/ToFFi_Toolbox-20211013/DATA_PREPARATION/HCP_DATA_PREP/output/000000_RENAMED/Sub_1


In [3]:
!pwd

/Users/jan/Documents/Studia/AdvancedProgramming/project/PyToFFi


In [5]:
class DataWrap:
    '''Class for loading data in matlab format
    
    - Obligatory data structures with fields:
        - Data: self.sig['data']
            - trial 
            - time
            - label
            - fsample
        - spatial filter: self.lcmv['spatialFilter']
        - brain atlas
            - sourceAtlas:
                - dim: [20 25 22]
                - transform: [4×4 double] 
                - unit: 'cm'
                - tissue: [20×25×22 double]
                - tissuelabel: {1×100 cell}
        - common source grid = sourcemodel = template grid = commonData/templategrid_HCP_8mm.mat
            - xgrid
            - ygrid
            - zgrid
            - dim
            - pos
            - inside
            - outside
    
    '''
    
    
    def __init__(self, data_dir: str, sub: int, sourcemodel: str):
        self.data_dir = data_dir #loading the data directory path
        self.cov = loadmat(f'{data_dir}/Sub_{sub}/data_clean_covmat_HCP_att2_{sub}.mat', simplify_cells=True) #loading covariance matrix
        self.sig = loadmat(f'{data_dir}/Sub_{sub}/data_clean_HCP_att2_{sub}.mat', simplify_cells=True) #loading data frame - MGE signal etc.
        self.lcmv = loadmat(f'{data_dir}/Sub_{sub}/flt_LCMV_HCP_att2_{sub}.mat', simplify_cells=True) #loading lcmv filter
        self.template = loadmat(f'{template_dir}', simplify_cells=True)
        
        self.data = self.sig['data']
        
        # self.fsample = self.sig['data']['fsample']
        # self.trial = self.sig['data']['trial']
        # self.time = self.sig['data']['time']
        # self.label = self.sig['data']['label']
        
        self.atl = loadmat(atlas_dir, simplify_cells=True)
        self.atlas = self.atl['sourceAtlas']
        # self.dim = self.atlas['dim']#[0][0][0][0]
        # self.transform = self.
        
        self.sourcemodel = self.template['sourcemodel']


In [6]:
sub_3 = DataWrap(data_dir=data_dir, sub=3, sourcemodel=template_dir)

In [7]:
sub_3.lcmv['spatialFilter'].shape

(5798, 245)

1. co to są cell array w matlabie - wielowymiarowa lista
2. array w matlab - przechowuje tylko jeden typ
3. supfunsim - paper - klasy itd

4. rm_output_data - usuwanie outputi
5. skrypt integrate

6. STAGE 1 source projection 

7. Beamforming - lcmv

8. Use group atlas from commonData 

sub_1.data['hdr']['Fs'] #original sampling rate = 2034...

sub_1.sig['data']['fsample'] # GO TO reduced sampling rate for computations = 508...

9. Mnożenie '*' w Matlabie to matrix multip. '.*' 

sub_3.data['trial'][0].shape #badani mogą mieć różną liczbę kanałów, 509 próbek sygnału powinno być dla każdego

(245, 509)

CFG.(STAGE_NAME).frequenciesOfInterest = logspace(0, log10(40), 20);

In [8]:
def sourceProjection(data, lcmv):
    
# """  
#   Calculates product of data matrix and spatial filter matrix in order to
# perform source activity reconstruction.

# More on the topic:
# [1] K. Sekihara and S. S. Nagarajan, Adaptive Spatial Filters for
#     Electromagnetic Brain Imaging. Berlin Heidelberg: Springer-Verlag, 2008.
# % Inputs
# *cfg* - [] (empty) in current version.

# *data* - structure containing following fields:

# _trial_ - cell containing 2D-arrays (sensors x time frames) signal values for
# single epochs of the multichannel signal (time segments).

# _time_ - cell containing 1D-arrays (1 x time frames) containing time
# values for single epochs of the multichannel signal (time segments).

# _label_ - cell containing names of the sensors (e.g. EEG electodes, MEG squids).

# _fsample_ - double; sampling frequency in Hz

# *spatialFilter* - double; 2D-array which represents spatial filter
#                   coefficients used for reconstruction on brain activity
#                   from sensors signal (see e.g. Van Veen 1997 to learn
#                   about LCMV spatial filter).
# % Outputs
# *sourceTimeSeries* - structure containing following fields:

# _trial_ - cell containing 2D-arrays (sources x time frames) reconstructed
# source activity signal values for single epochs (time segments).

# _time_ - cell containing 1D-arrays (1 x time frames) containing time
# values for single epochs of the reconstructed activity signal.

# _fsample_ - double; sampling frequency in Hz

# _label_ - cell containing working names of the sources
#           ({'src_1', 'src_2', ..., 'src_5998}).
#   """


    print('Source projection ...')
    sourceTimeSeries = {}
# SOURCE PROJECTION: apply weights to a sensor signal
    numTrials                    = len(data['trial'])
    numSources                   = lcmv['spatialFilter'].shape[0]
    sourceTimeSeries['trial']    = np.empty(shape=(1, numTrials), dtype=object)
    sourceTimeSeries['time']     = data['time']
    sourceTimeSeries['fsample']  = data['fsample']
    sourceTimeSeries['label']    = np.empty((numSources, 1),dtype=object) # labels are needed to perform ft_freqanalysis

    for src in range(0,numSources):
        sourceTimeSeries['label'][src] = sourceTimeSeries['label'][src]
        sourceTimeSeries['label'][src] = 'src_'+str(src)
    

    # # source projection: q = Wy
    for tt in range(0,numTrials): #CHANGED FROM 1-indexing IN MATLAB
        sourceTimeSeries['trial'][0][tt] = np.matmul(lcmv['spatialFilter'],data['trial'][tt])
    

    if 'sampleinfo' in data.keys():
        sourceTimeSeries['sampleinfo'] = data['sampleinfo']
    

    return sourceTimeSeries


In [9]:
data_src = sourceProjection(sub_3.data, sub_3.lcmv)

Source projection ...


# Frequency analysis with syncopy

In [36]:
import math
from scipy.signal import windows


In [37]:
print('Calculating power spectra for each time segment ...')

cfg = {}
cfg['frequencies'] = np.logspace(0,math.log(40, 10),num=20)



Calculating power spectra for each time segment ...


In [38]:
x = windows.dpss(,)

array([1.])

In [28]:
cfgFreq = {}


cfgFreq['method'] = 'mtmfft'
cfgFreq['output'] = 'pow'
cfgFreq['taper']  = 'dpss'
cfgFreq['tapsmofrq'] = 2
cfgFreq['pad'] = 2
cfgFreq['keeptrials'] = 'yes'
cfgFreq['foi'] = cfg['frequencies']

# powSpectra  = ft_freqanalysis(cfgFreq, timeSeries);
# outputFrequencies   = powSpectra.freq;


20:07:25 - CRITICAL: 
SyNCoPy encountered an error in 

/var/folders/hr/8qs4_3tn5tb9cg_2gmnz7pc00000gn/T/ipykernel_3179/720840815.py, line 10 in <module>
	cfgFreq.data        = spy.AnalogData(data_src)

--------------------------------------------------------------------------------
Abbreviated traceback:

/Users/jan/anaconda3/lib/python3.10/site-packages/syncopy/datatype/continuous_data.py, line 401 in __init__
	super().__init__(data=data,
/Users/jan/anaconda3/lib/python3.10/site-packages/syncopy/datatype/continuous_data.py, line 317 in __init__
	super().__init__(data=data, **kwargs)
/Users/jan/anaconda3/lib/python3.10/site-packages/syncopy/datatype/base_data.py, line 1458 in __init__
	setattr(self, propertyName, kwargs[propertyName])
/Users/jan/anaconda3/lib/python3.10/site-packages/syncopy/datatype/continuous_data.py, line 66 in data
	self._set_dataset_property(inData, "data")
/Users/jan/anaconda3/lib/python3.10/site-packages/syncopy/datatype/base_data.py, line 293 in _set_dataset_p